## Action items
1. spark read delta table from adls, use expression to group by for latestchanges.
2. call write api from adb to synapse sql dedicated pool to store latestchanges.
3. auth to sql pool using keyvault

In [4]:
val df_delta = spark.read.format("delta").load("abfss://wplushiramsynapsefs@wplushiramsynapseadlsv2.dfs.core.windows.net/synapse/workspaces/wplushiramsynapse/warehouse/lastchanges")
df_delta.show()

In [9]:
val latestonly = df_delta.selectExpr("key", "struct(time, newValue, deleted) as otherCols").groupBy("key").agg(max("otherCols").as("latest")).selectExpr("key", "latest.*")
latestonly.show()

In [10]:
latestonly.createOrReplaceTempView("latestonly")

In [11]:
%%sql
select * from latestonly

In [12]:
latestonly.write.option(Constants.SERVER, "wplushiramsynapse.sql.azuresynapse.net").synapsesql("wplussynapsedw.dbo.latestonly", Constants.INTERNAL)

## But he wants it in ADB.
1. https://docs.databricks.com/data/data-sources/azure/synapse-analytics.html#frequently-asked-questions-faq
2. https://github.com/Azure-Samples/azure-sql-db-databricks/blob/main/notebooks/03b-parallel-switch-in-load-into-partitioned-table-single.ipynb